### Creates a Pandas DataFrame of Ocetrac attributes 


In [2]:
import xarray as xr
import ocetrac

In [3]:
file_path_list = ('/burg/home/hs3277/marine-heatwaves/data/preprocessed_oisst.nc',
                  '/burg/home/hs3277/marine-heatwaves/data/ocetrac_labels.nc')

ds = xr.open_mfdataset(file_path_list)
ssta = ds.ssta_notrend
labels = ds.blobs
num_events = labels.attrs['final objects tracked']
print(f"There are {num_events} total events tracked.")
ds

There are 779 total events tracked.


<xarray.Dataset>
Dimensions:       (time: 478, lat: 720, lon: 1440)
Coordinates:
  * lon           (lon) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * time          (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2021-06-01
Data variables:
    blobs         (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    lsmask        (lat, lon) float32 dask.array<chunksize=(720, 1440), meta=np.ndarray>
    mean          (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    trend         (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    seas          (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_notrend  (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
Attributes:
    description:  Ocetrac labels
    data:         OISST v2.1 preprocessed for Ocetrac
    threshold:    90th percentile
    climatology:  entire period

In [4]:
%%time
labels.load();

CPU times: user 0 ns, sys: 2.42 s, total: 2.42 s
Wall time: 2.53 s


<xarray.DataArray 'blobs' (time: 478, lat: 720, lon: 1440)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * time     (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2021-06-01
Attributes:
    inital objects identified:  12704
    final objects tracked:      779
    radius:                     8
    size quantile threshold:    0.75
    min area:                   2487.0
    percent area reject:        0.19759675219466732
    percent area accept:        0.8024032478053327

### Section of code in Ocetrac `metrics.py`

In [5]:
import os, glob
import xarray as xr
import numpy as np
import pandas as pd
import warnings
from skimage.measure import label as label_np
from skimage.measure import regionprops 

try:
    from tqdm.auto import tqdm
    tqdm_avail = True
except:
    warnings.warn(
        "Optional dependency `tqdm` not found. This will make progressbars a lot nicer. \
    Install with `conda install -c conda-forge tqdm`"
    )
    tqdm_avail = False
    
def _wrap(labels):
    ''' Impose periodic boundary and wrap labels, then reorder the labels'''
    first_column = labels[..., 0]
    last_column = labels[..., -1]

    stacked = first_column.stack(z=['time','lat'])
    unique_first = np.unique(stacked[stacked.notnull()])

    # This loop iterates over the unique values in the first column, finds the location of those values in 
    # the first columnm and then uses that index to replace the values in the last column with the first column value
    for i in enumerate(unique_first):
        first = np.where(first_column == i[1])
        last = last_column[first[0], first[1]]
        stacked = last.stack(z=['time','lat'])
        bad_labels = np.unique(stacked[stacked.notnull()])
        replace = np.isin(labels, bad_labels)
        labels = labels.where(replace==False, other=i[1])
    
    labels = labels.fillna(0)
    labels_wrapped = np.unique(labels, return_inverse=True)[1].reshape(labels.shape)
    labels_wrapped = xr.DataArray(labels_wrapped, dims=labels.dims, coords=labels.coords)

    return labels_wrapped

def _get_labels(binary_images):
    '''function used to label binary images at each time step using skimage.measure.label'''
    blobs_labels = label_np(binary_images, background=0)
    return blobs_labels
    
def _get_centroids(sub_labels):
    '''This function uses skimage.measure.regionprops to find the centroids of objects assigned 
    to each unique label'''
    props = regionprops(sub_labels.astype('int'))
    centroids = [(float(sub_labels.lat[round(p.centroid[0])].values),
                  float(sub_labels.lon[round(p.centroid[1])].values)) for p in props]
    for i in range(0,len(centroids)):
        if centroids[i][1] >= 359.875:
            centroids[i] = (centroids[i][0], list(centroids[i])[1] - 359.875)
    
    return centroids

def _get_intensity_area(event, ssta, mhw):
    '''Calculates event intensities and area at each time interval using anomaly data and coordinates 
    cooresponding to the event.'''
    
    event_ssta = ssta.where(event>0, drop=True)        
    mhw['intensity_mean'].append(event_ssta.mean(('lat','lon')).values)
    mhw['intensity_max'].append(event_ssta.max(('lat','lon')).values) 
    mhw['intensity_min'].append(event_ssta.min(('lat','lon')).values)
    mhw['intensity_cumulative'].append(np.nansum(event_ssta))
    coords = event.stack(z=('lat','lon'))
    coord_pairs = [(coords.isel(time=t[0]).dropna(dim='z', how='any').z.lat.values, 
                      coords.isel(time=t[0]).dropna(dim='z', how='any').z.lon.values) for t in enumerate(event.time)]

    mhw['coords'].append(coord_pairs)

    # Calculate weighted cell area assuming 0.25º resolution data and 111 km per degree of latitude
    y, x = zip(*coord_pairs)
    dlon = [np.cos(y[c]*np.pi/180)*(111*.25) for c in np.arange(0, len(coord_pairs))]
    dlat = (111*.25) * np.ones(len(dlon))
    cell_area = [np.sum(dlon[c]*dlat[c]) for c in np.arange(0, len(coord_pairs))]
    mhw['area'].append(cell_area)

    return mhw
    


In [6]:
def _get_metrics(event, ssta):
    '''
    Creates a Pandas DataFrame of event attributes.
    
    Parameters
    ----------
      event : xarray.DataArray   
              Image set containing only objects corresponding to the event of interest. 
              Dimensions should be ('time', 'lat', 'lon')
              
      ssta  : xarray.DataArray
              Temperature vector [1D numpy array of length T]
    
    Returns
    -------
    
    mhw : pandas.DataFrame
          Marine heat wave event attributes. The keys listed below are 
          are contained within the dataset.
 
        'id'                     Unique label given to the MHW [int]
        'date'                   Dates corresponding to the event [datetime format]
        'coords'                 Latitude and longitude of all points contained in the event [tuple(lat,lon)]
        'centroid'               Center of each object contained in the event [tuple(lat,lon)]
        'duration'               Duration of event [months]
        'intensity_max'          Maximum intensity at each time interval [degC]
        'intensity_mean'         Mean intensity at each time interval [degC]
        'intensity_min'          Minimum intensity at each time interval [degC]
        'intensity_cumulative'   Cumulated intensity over the entire event [degC months]
        'area'                   Area of the event at each time interval [km2]
        
    '''
    
    # Initialize dictionary 
    mhw = {}
    mhw['id'] = [] # event label
    mhw['date'] = [] # datetime format
    mhw['coords'] = [] # (lat, lon)
    mhw['centroid'] = []  # (lat, lon)
    mhw['duration'] = [] # [months]
    mhw['intensity_max'] = [] # [deg C]
    mhw['intensity_mean'] = [] # [deg C]
    mhw['intensity_min'] = [] # [deg C]
    mhw['intensity_cumulative'] = [] # [deg C]
    mhw['area'] = [] # [km2]

    # TO ADD:
    # mhw['rate_onset'] = [] # [deg C / month]
    # mhw['rate_decline'] = [] # [deg C / month]

    mhw['id'].append(int(np.nanmedian(event.values)))
    mhw['date'].append(event.time.values.astype('datetime64[M]'))
    mhw['duration'].append(event.time.shape[0])

    # Turn images into binary
    binary_event = event.where(event>=0, other=0)
    binary_event = binary_event.where(binary_event==0, other=1)
      
    sub_labels = xr.apply_ufunc(_get_labels, binary_event,
                                input_core_dims=[['lat', 'lon']],
                                output_core_dims=[['lat', 'lon']],
                                output_dtypes=[binary_event.dtype],
                                vectorize=True,
                                dask='parallelized')
    
    # Turn background to NaNs
    sub_labels = xr.DataArray(sub_labels, dims=binary_event.dims, coords=binary_event.coords)
    sub_labels = sub_labels.where(sub_labels>0, drop=False, other=np.nan) 

    # The labels are repeated each time step, therefore we relabel them to be consecutive
    for p in range(1, sub_labels.shape[0]):
        sub_labels[p,:,:] = sub_labels[p,:,:].values + sub_labels[p-1,:,:].max().values
    
    sub_labels_wrapped = _wrap(sub_labels)
    
    mhw = _get_intensity_area(event, ssta, mhw)
    
    centroid = []
    for s in np.arange(0, sub_labels_wrapped.shape[0]):
        lx = sub_labels_wrapped.isel(time=s)
        east = lx.where(lx.lon < 180, drop=True)
        east['lon'] = np.arange(360.125, 540.125, .25)
        append_east = xr.concat([lx.where(lx.lon >= 180, drop=True), east], dim="lon")
        centroid.append(_get_centroids(append_east))
    mhw['centroid'].append(centroid)
    
    mhw = pd.DataFrame(dict([(name, pd.Series(data)) for name,data in mhw.items()]))
#     mhw.to_csv('df_'+str(mhw['id'].values[0]).zfill(4)+'.csv', index=False)
    return mhw




In [7]:
num_events = int(np.nanmax(labels))
num_events

779

In [26]:
df.append(df)

,id,date,coords,centroid,duration,intensity_max,intensity_mean,intensity_min,intensity_cumulative,area
0,779,[2021-06],"[([-46.375, -46.375, -46.375, -46.375, -46.375...","[[(-33.625, 207.875)]]",1,[1.6614285951021799],[0.618134235727127],[-0.33177519602952543],2976.316345,[3064032.7501829183]
0,779,[2021-06],"[([-46.375, -46.375, -46.375, -46.375, -46.375...","[[(-33.625, 207.875)]]",1,[1.6614285951021799],[0.618134235727127],[-0.33177519602952543],2976.316345,[3064032.7501829183]


#### How do we parrelize this instead of looping over event ID

In [22]:
dataframes = []  
for i in range(1,num_events+1):
    print(i)
    event = labels.where(labels==i, drop=True).reindex({"lon": labels.lon.values})
    df = _get_metrics(event, ssta)
    dataframes.append(df)

1


/burg/home/hs3277/.conda/envs/marine-heatwaves/lib/python3.9/site-packages/xarray/core/indexing.py:1232: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


2


/burg/home/hs3277/.conda/envs/marine-heatwaves/lib/python3.9/site-packages/xarray/core/indexing.py:1232: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


3
4
5
6


...

767
768


/burg/home/hs3277/.conda/envs/marine-heatwaves/lib/python3.9/site-packages/xarray/core/indexing.py:1232: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


769
770
771
772
773


/burg/home/hs3277/.conda/envs/marine-heatwaves/lib/python3.9/site-packages/xarray/core/indexing.py:1232: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


774
775
776
777
778
779


In [ ]:
# def to_dataframe(blobs, ssta):

#     directory = os.getcwd()+'/ocetrac_csv/'
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     os.chdir(directory)
    
#     num_events = int(np.nanmax(blobs)) # number of events from Ocetrac
#     print(f'CSV files are saved here: {directory}')
#     for i in tqdm(range(1,num_events+1)):
#         event = blobs.where(blobs==i, drop=True).reindex({"lon": blobs.lon.values})
#         _save_csv(event, ssta)
    
#     csvfiles = glob.glob(os.path.join(directory, '*.csv'))
#     dataframes = []  
#     for csvfile in csvfiles:
#         df = pd.read_csv(csvfile)
#         dataframes.append(df)
    
#     df = pd.concat(dataframes, ignore_index=True)
    
#     for f in glob.glob(os.path.join(directory, '*.csv')):
#         if os.path.exists(f):
#             os.remove(f)
#     os.rmdir(directory)

#     return df

In [ ]:
# df = ocetrac.to_dataframe(labels, ssta)

In [33]:
dff = pd.concat(dataframes, ignore_index=True)

In [34]:
dff

,id,date,coords,centroid,duration,intensity_max,intensity_mean,intensity_min,intensity_cumulative,area
0,1,"[1981-09, 1981-10, 1981-11, 1981-12, 1982-01, ...","[([-63.875, -63.875, -63.875, -63.875, -63.875...","[[(-54.625, 24.0), (-1.625, 138.75), (48.625, ...",37,"[3.440791046728215, 2.6536470312559324, 2.7766...","[0.8432808915179753, 0.7900916980703073, 0.692...","[-1.114350739540976, -1.121826228737504, -2.07...",1.618654e+06,"[25016235.227460224, 20314532.85214622, 166710..."
1,2,"[1981-09, 1981-10]","[([-61.125, -61.125, -61.125, -61.125, -61.125...","[[(-50.375, 52.5)], [(-42.875, 6.25)]]",2,"[3.8220915271007962, 3.192730869297744]","[0.5819077331149765, 0.8880185892117968]","[-1.626915785464993, -0.0354472760039205]",1.210291e+04,"[6307174.727972034, 2879390.507268061]"
2,3,[1981-09],"[([-54.875, -54.875, -54.875, -54.875, -54.875...","[[(-46.125, 329.625)]]",1,[2.550364092287726],[0.7339658124806407],[-1.21935854947154],4.076446e+03,[2953236.170290635]
3,4,[1981-09],"[([-27.125, -27.125, -27.125, -27.125, -27.125...","[[(-19.375, 195.625)]]",1,[1.8420186457123258],[0.7858445913476392],[-0.5460597827348508],2.343389e+03,[2162179.7983823]
4,5,"[1981-09, 1981-10]","[([-9.375, -9.375, -9.375, -9.375, -9.375, -9....","[[(-4.125, 75.5)], [(-5.125, 85.5)]]",2,"[1.282908240694855, 1.8585829802973954]","[0.570964357074938, 0.5716692469226826]","[-0.2816908070977746, -0.5411240934625638]",5.094167e+03,"[2465356.949916051, 4354387.439564242]"
...,...,...,...,...,...,...,...,...,...,...
774,775,"[2021-05, 2021-06]","[([0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0...","[[(10.625, 89.75)], [(14.375, 89.25)]]",2,"[1.5356856495282685, 1.6816798704415206]","[0.6993772437283774, 0.6951061741674927]","[0.14332847551571604, -0.07546848921989735]",4.356382e+03,"[2724569.3636955507, 1957682.7295617382]"
775,776,[2021-05],"[([33.125, 33.125, 33.125, 33.125, 33.125, 33....","[[(38.625, 212.875)]]",1,[2.369298414600676],[1.360129948382017],[0.41517881977680915],5.195696e+03,[2294821.0290892674]
776,777,[2021-06],"[([-60.875, -60.875, -60.875, -60.875, -60.875...","[[(-51.875, 89.5)]]",1,[3.258678933045766],[0.4564444320817318],[-2.5181767013199394],2.545591e+03,[2642573.0410941066]
777,778,[2021-06],"[([-47.125, -47.125, -47.125, -47.125, -47.125...","[[(-37.625, 111.75)]]",1,[3.79459194541138],[0.6827753929692493],[-1.3547077051537695],2.881312e+03,[2564312.899001384]


### Save metrics as CSV

In [35]:
dff.to_csv('/burg/home/hs3277/marine-heatwaves/data/ocetrac_metrics.csv', index=False)


#### Event Attributes
        
- **id** : Unique label given to the MHW [int]
- **date** : Dates corresponding to the event [datetime format]
- **coords** : Latitude and longitude of all points contained in the event [(lat,lon)]
- **centroid** : Center of each object contained in the event [(lat,lon)]
- **duration** : Duration of event [months]
- **intensity_max** : Maximum intensity at each time interval [degC]
- **intensity_mean** : Mean intensity at each time interval [degC]
- **intensity_min** : Minimum intensity at each time interval [degC]
- **intensity_cumulative** : Cumulated intensity over the entire event [degC months]
- **area** : Area of the event at each time interval [km2]

TO ADD?
- number of centroids
- maximum distance between centroids
- date peak [datetime format]
- rate onset [degC / month]
- rate decline [degC / month]